# Example 1.1
This example follows the `examples/1.1`  
Search for a simple lj crystal with 8 numbers percell

In [112]:
from airss_ase import SeedAtoms, Buildcell
from ipypb import ipb as tqdm

# Import ase assets
from ase.optimize import BFGSLineSearch, sciopt, FIRE, BFGS, precon
from ase.calculators.lj import LennardJones
from ase.constraints import UnitCellFilter, ExpCellFilter

In [34]:
# Define the search seed

seed = SeedAtoms('Al', cell=[2, 2, 2], pbc=True)
seed.gentags.minsep = 1.5
# Define per-atom tags, here it is simply request 8 Al atoms
al = seed[0]
al.num = 8

In [36]:
# LJ potential calculator
lj = LennardJones(signma=2, epsilon=1, beta=1, rc=2.5)

In [140]:
# The seed in text form
print('\n'.join(seed.get_cell_inp_lines()))

%BLOCK lattice_cart
2.0000000000  0.0000000000  0.0000000000
0.0000000000  2.0000000000  0.0000000000
0.0000000000  0.0000000000  2.0000000000
%ENDBLOCK lattice_cart
%BLOCK positions_abs
Al  0.0000000000 0.0000000000 0.0000000000 # Al0 % NUM=8
%ENDBLOCK positions_abs
#MINSEP=1.5



## Generate the random structure
The random structrue can be built by calling `build_random_atoms` method.
An `ase.Atoms` object is returned.

In [126]:
rand_atoms = seed.build_random_atoms()

Inpsect the cell

In [127]:
rand_atoms.get_cell()

array([[4.25031   , 0.        , 0.        ],
       [1.81680255, 4.00673793, 0.        ],
       [1.30959483, 1.82009312, 3.89759852]])

Inspect the cell volume
The initial cell volume is $2\times2\times2=8$ and we requested 8 atoms.
So the target cell volume is 64 but `buildcell` will scale to a random value
within 50%.

In [128]:
rand_atoms.get_volume()

66.37562845079496

In [129]:
rand_atoms.get_positions()

array([[2.13198482, 3.13673947, 3.85723496],
       [1.67108971, 4.02214929, 2.73349501],
       [3.95796943, 3.83131731, 3.38519762],
       [3.50212198, 3.14885171, 5.5703684 ],
       [1.83124691, 2.96549873, 5.37154553],
       [0.81827181, 0.98508932, 1.95748114],
       [0.73402131, 2.48557134, 4.37771662],
       [3.52906852, 4.59171708, 6.14269305]])

## Do the search
The search can be carried out directly using the calculator and optimizers in `ase` 

In [131]:
def gen_and_relax(seed, calc, opti):
    """
    Generate and relax the cell
    """
    rand_atoms = seed.build_random_atoms()
    rand_atoms.set_pbc(True)  # Set the PBC as we are search from crystal
    rand_atoms.set_calculator(calc)  # Attached hte calculator
    # Apply the UnitCellFilter as we are also optimizing the cell
    opt = opti(UnitCellFilter(rand_atoms), logfile=None) 
    opt.run(fmax=0.05)
    return rand_atoms

In [132]:
res = []
for i in tqdm(range(20)):
    # A range of the possible optimization algorithm are possible
    relaxed = gen_and_relax(atoms, lj, BFGSLineSearch)
    res.append(relaxed)

 [████████████████████████████████████████████████████████████] 20/20 [00:38<00:02, 1.88s/it]

In [134]:
## Get the symmetry of the relaxed structures

In [133]:
from spglib import get_spacegroup
spg = [get_spacegroup(i, 0.5) for i in res]
spg

['Fm-3m (225)',
 'Cm (8)',
 'Fm-3m (225)',
 'C2/m (12)',
 'Fm-3m (225)',
 'P6_3/mmc (194)',
 'Pmma (51)',
 'P6_3/mmc (194)',
 'P6_3/mmc (194)',
 'P4/nmm (129)',
 'Fm-3m (225)',
 'I4/mmm (139)',
 'Fm-3m (225)',
 'Fm-3m (225)',
 'Fm-3m (225)',
 'P6_3/mmc (194)',
 'P6_3/mmc (194)',
 'Cm (8)',
 'Fm-3m (225)',
 'C2/m (12)']

## Alternatively, use the native airss script to run the search

In [135]:
!mkdir work

mkdir: cannot create directory ‘work’: File exists


In [136]:
%%timeit -n 1 -r 1
seed.write_seed('work/Al.cell')
!echo "1 12 6 2.5\nAl\n# Epsilon\n1\n# Sigma\n2\n# Beta\n1" >> work/Al.pp
!cd work && airss.pl -seed Al -max 20 -pp3 && cd ../

10.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Note that the `pp3` program for relxation will run significantly faster than doing so in python

In [137]:
!cd work/ &&  ca -r -u 0.01 && cd -

Al-8858-7126-18          0.00     0.945     -13.318   8 Al           P63/mmc    1
Al-8858-7126-1           0.00     0.945       0.000   8 Al           P63/mmc    1
Al-8858-7126-10          0.00     0.945       0.010   8 Al           Fm-3m      1
Al-12455-1866-20        -0.00     0.945       0.010   8 Al           Fm-3m      1
Al-12455-1866-8         -0.00     1.059       1.637   8 Al           P-1        1
Al-12455-1866-15        -0.00     1.683       2.172   8 Al           C2/m       1
Al-5573-1513-16         -0.00     1.713       2.172   8 Al           C2/m       1
Al-5573-1513-3           0.00     1.838       2.397   8 Al           Cmcm       1
Al-12455-1866-19         0.00     1.849       2.397   8 Al           P21/m      1
Al-8858-7126-20          0.00     1.897       3.248   8 Al           Cm         1
Al-8858-7126-2          -0.00     2.009       3.597   8 Al           Cm         1
Al-8858-7126-5           0.00     2.732       3.646   8 Al           Cm         1
Al-5573-1513-10 